# Classificatore di Immagini Semplice

Questo notebook ti mostra come classificare immagini utilizzando una rete neurale pre-addestrata.

**Cosa imparerai:**
- Come caricare e utilizzare un modello pre-addestrato
- Pre-elaborazione delle immagini
- Fare previsioni sulle immagini
- Comprendere i punteggi di confidenza

**Caso d'uso:** Identificare oggetti nelle immagini (come "gatto", "cane", "auto", ecc.)

---


## Passaggio 1: Importa le librerie necessarie

Importiamo gli strumenti di cui abbiamo bisogno. Non preoccuparti se non comprendi ancora tutti questi!


In [ ]:
# Core libraries
import numpy as np
from PIL import Image
import requests
from io import BytesIO

# TensorFlow for deep learning
try:
    import tensorflow as tf
    from tensorflow.keras.applications import MobileNetV2
    from tensorflow.keras.applications.mobilenet_v2 import preprocess_input, decode_predictions
    print("✅ TensorFlow loaded successfully!")
    print(f"   Version: {tf.__version__}")
except ImportError:
    print("❌ Please install TensorFlow: pip install tensorflow")

## Passaggio 2: Carica il Modello Pre-addestrato

Utilizzeremo **MobileNetV2**, una rete neurale già addestrata su milioni di immagini.

Questo si chiama **Transfer Learning** - utilizzare un modello che qualcun altro ha già addestrato!


In [ ]:
print("📦 Loading pre-trained MobileNetV2 model...")
print("   This may take a minute on first run (downloading weights)...")

# Load the model
# include_top=True means we use the classification layer
# weights='imagenet' means it was trained on ImageNet dataset
model = MobileNetV2(weights='imagenet', include_top=True)

print("✅ Model loaded!")
print(f"   The model can recognize 1000 different object categories")

## Passaggio 3: Funzioni di Supporto

Creiamo funzioni per caricare e preparare le immagini per il nostro modello.


In [ ]:
def load_image_from_url(url):
    """
    Load an image from a URL.
    
    Args:
        url: Web address of the image
        
    Returns:
        PIL Image object
    """
    response = requests.get(url)
    img = Image.open(BytesIO(response.content))
    return img


def prepare_image(img):
    """
    Prepare an image for the model.
    
    Steps:
    1. Resize to 224x224 (model's expected size)
    2. Convert to array
    3. Add batch dimension
    4. Preprocess for MobileNetV2
    
    Args:
        img: PIL Image
        
    Returns:
        Preprocessed image array
    """
    # Resize to 224x224 pixels
    img = img.resize((224, 224))
    
    # Convert to numpy array
    img_array = np.array(img)
    
    # Add batch dimension (model expects multiple images)
    img_array = np.expand_dims(img_array, axis=0)
    
    # Preprocess for MobileNetV2
    img_array = preprocess_input(img_array)
    
    return img_array


def classify_image(img):
    """
    Classify an image and return top predictions.
    
    Args:
        img: PIL Image
        
    Returns:
        List of (class_name, confidence) tuples
    """
    # Prepare the image
    img_array = prepare_image(img)
    
    # Make prediction
    predictions = model.predict(img_array, verbose=0)
    
    # Decode predictions to human-readable labels
    # top=5 means we get the top 5 most likely classes
    decoded = decode_predictions(predictions, top=5)[0]
    
    # Convert to simpler format
    results = [(label, float(confidence)) for (_, label, confidence) in decoded]
    
    return results


print("✅ Helper functions ready!")

## Passaggio 4: Test su immagini di esempio

Proviamo a classificare alcune immagini prese da internet!


In [ ]:
# Sample images to classify
# These are from Unsplash (free stock photos)
test_images = [
    {
        "url": "https://images.unsplash.com/photo-1514888286974-6c03e2ca1dba?w=400",
        "description": "A cat"
    },
    {
        "url": "https://images.unsplash.com/photo-1552053831-71594a27632d?w=400",
        "description": "A dog"
    },
    {
        "url": "https://images.unsplash.com/photo-1511919884226-fd3cad34687c?w=400",
        "description": "A car"
    },
]

print(f"🧪 Testing on {len(test_images)} images...")
print("=" * 70)

### Classifica ogni immagine


In [ ]:
for i, img_data in enumerate(test_images, 1):
    print(f"\n📸 Image {i}: {img_data['description']}")
    print("-" * 70)
    
    try:
        # Load image
        img = load_image_from_url(img_data['url'])
        
        # Display image
        display(img.resize((200, 200)))  # Show smaller version
        
        # Classify
        results = classify_image(img)
        
        # Show predictions
        print("\n🎯 Top 5 Predictions:")
        for rank, (label, confidence) in enumerate(results, 1):
            # Create a visual bar
            bar_length = int(confidence * 50)
            bar = "█" * bar_length
            
            print(f"  {rank}. {label:20s} {confidence*100:5.2f}% {bar}")
        
    except Exception as e:
        print(f"❌ Error: {e}")

print("\n" + "=" * 70)

## Passaggio 5: Prova con le tue immagini!

Sostituisci l'URL qui sotto con qualsiasi URL di immagine che desideri classificare.


In [ ]:
# Try your own image!
# Replace this URL with any image URL
custom_image_url = "https://images.unsplash.com/photo-1472491235688-bdc81a63246e?w=400"  # A flower

print("🖼️  Classifying your custom image...")
print("=" * 70)

try:
    # Load and show image
    img = load_image_from_url(custom_image_url)
    display(img.resize((300, 300)))
    
    # Classify
    results = classify_image(img)
    
    # Show results
    print("\n🎯 Top 5 Predictions:")
    print("-" * 70)
    for rank, (label, confidence) in enumerate(results, 1):
        bar_length = int(confidence * 50)
        bar = "█" * bar_length
        print(f"  {rank}. {label:20s} {confidence*100:5.2f}% {bar}")
    
    # Highlight top prediction
    top_label, top_confidence = results[0]
    print("\n" + "=" * 70)
    print(f"\n🏆 Best guess: {top_label} ({top_confidence*100:.2f}% confident)")
    
except Exception as e:
    print(f"❌ Error: {e}")
    print("   Make sure the URL points to a valid image!")

## 💡 Cosa è appena successo?

1. **Abbiamo caricato un modello pre-addestrato** - MobileNetV2 è stato addestrato su milioni di immagini  
2. **Abbiamo preprocessato le immagini** - Ridimensionate e formattate per il modello  
3. **Il modello ha fatto previsioni** - Ha fornito probabilità per 1000 classi di oggetti  
4. **Abbiamo decodificato i risultati** - Convertito i numeri in etichette comprensibili  

### Comprendere i punteggi di confidenza

- **90-100%**: Molto sicuro (quasi certamente corretto)  
- **70-90%**: Sicuro (probabilmente corretto)  
- **50-70%**: Moderatamente sicuro (potrebbe essere corretto)  
- **Sotto il 50%**: Poco sicuro (incerto)  

### Perché le previsioni potrebbero essere sbagliate?

- **Angolazione o illuminazione insolita** - Il modello è stato addestrato su foto tipiche  
- **Oggetti multipli** - Il modello si aspetta un oggetto principale  
- **Oggetti rari** - Il modello conosce solo 1000 categorie  
- **Immagine di bassa qualità** - Immagini sfocate o pixelate sono più difficili da analizzare  

---


## 🚀 Prossimi Passi

1. **Prova immagini diverse:**
   - Trova immagini su [Unsplash](https://unsplash.com)
   - Clic destro → "Copia indirizzo immagine" per ottenere l'URL

2. **Sperimenta:**
   - Cosa succede con l'arte astratta?
   - Può riconoscere oggetti da angolazioni diverse?
   - Come gestisce più oggetti contemporaneamente?

3. **Approfondisci:**
   - Esplora le [lezioni di Visione Artificiale](../lessons/4-ComputerVision/README.md)
   - Impara a creare il tuo classificatore di immagini
   - Comprendi come funzionano le CNN (Reti Neurali Convoluzionali)

---

## 🎉 Congratulazioni!

Hai appena costruito un classificatore di immagini utilizzando una rete neurale all'avanguardia!

Questa stessa tecnica alimenta:
- Google Foto (organizzazione delle tue foto)
- Auto a guida autonoma (riconoscimento degli oggetti)
- Diagnosi medica (analisi delle radiografie)
- Controllo qualità (rilevamento dei difetti)

Continua a esplorare e imparare! 🚀



---

**Disclaimer**:  
Questo documento è stato tradotto utilizzando il servizio di traduzione automatica [Co-op Translator](https://github.com/Azure/co-op-translator). Sebbene ci impegniamo per garantire l'accuratezza, si prega di notare che le traduzioni automatiche possono contenere errori o imprecisioni. Il documento originale nella sua lingua nativa dovrebbe essere considerato la fonte autorevole. Per informazioni critiche, si raccomanda una traduzione professionale effettuata da un traduttore umano. Non siamo responsabili per eventuali incomprensioni o interpretazioni errate derivanti dall'uso di questa traduzione.
